In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis  

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

ModuleNotFoundError: No module named 'gensim'

In [ ]:
# Import data
df = pd.read_csv('data.csv')
df.head()

In [ ]:
# Preprocessing
# convert review text to string
df["review_text"] = df["review_text"].astype(str)
df["review_votes"] = df["review_votes"].astype(str)
df.review_text = df.review_text.apply(lambda s: s.strip())